In [2]:
training_data=[
    ['Green',3,'Mango'],
    ['Yellow',3,'Mango'],
    ['Red',1,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'lemon'],
]

In [8]:
#help(isinstance(7,int))

In [10]:
# column labels-- used to print the tree
header=["color","diameter","label"]

def unique_vals(rows,col):
    """finds the unique values for a column in a dataset"""
    return set([row[col] for row in rows])

#demo:
# unique_vals(training_data,0) -- finding unique values in terms of color
# unique_vals(training_data,0) -- finding unique values in terms of diameter
# set does not allow dupilicate values

def class_count(rows):
    """counts the no.of each type of example in a dataset"""
    counts={} # dictionary
    for row in rows:
        label=row[-1]
        if label not in counts:
            counts[label]=0
        else:
            counts[label]+=1
    return counts

def is_numeric(value):
    """Test if a value is numeric"""
    return isinstance(value,int) or isinstance(value,float) 

class Question:
    """a question is used to partition a dataset
    
    This class just records a column number (e.g., 0 for color) and a 
    column value (e.g., Green). The match method is used to compare the
    feature value in an example to the feature value stored in the question"""
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        # compare feature value in the example to the question class feature value
        val= example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        # this is a helper method to print
        condition="=="
        if is_numeric(self.value):
            condition=">="
        return "Is %s %s %s?" % (
        header[self.column], condition, str(self.value))
    
    
def partition(rows, question):
    """partitions a datset
    For each row in the dataset, check if it matches the question. If
    s0, add it to true rows, otherwise, add it to false rows.
    """
    
    true_rows, false_rows=[], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def gini(rows):
    """calculate the gini impurity for a list of rows"""
    counts = class_ounts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl =counts[lbl] / float(len(rows))
        impurity-=prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainity):
    p = float(len(left))/ (le(right)+len(left))
    return current_uncertainity - p*gini(left)-(1-p)*gini(right))
    
def find_best_spilt(rows):
    """find the best question to ask by iterating over every feature / value
    and calculating the information gain"""
    best_gain = 0 # keeps track of best information gain
    best_question= None # keep train of the feature/ value that produce it
    current_uncertainity = gini(rows)
    n_features = len(rows[0])-1 # number of columns
    
    for col in range(n_features):
        values = set([row[col] for row in rows])  # unique values in the column
        
        for val in values:
            
            question = Question(col, val)
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            #skip the split if it doesnot divide the dataset
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
                # calculate the information gain from the split
                
                gain= info_gain(true_rows, false_rows, current_uncertainity)
                
                # you actually can use > symbol instead of >= here
                # but i wanted the tree to look a certain way for our dataset
                
                if gain >= best_gain:
                    best_gain, best_question = gain, question
                    
                    
        return best_gain,best_question
    
class Leaf:
    """A leaf node classifies data.
    
    This holds a dictionary of class (eg., "mango") -> number of time
    it appears in the rows from the training data that reach this leaf"""
    
    def __init__(self,rows):
        self.predictions = class_counts(rows)
        
        
class Desicion_Node:
    """A Decision Node asks a question.
    This holds a references to the question, and to the two child nodes."""
    
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        
        
def build_tree(rows):
    """Builds the tree.
    
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). """
    
    # try partitioning the dataset on each of the unique attribute,
    # calculate the information gain
    # and return the question that produces the highest gain
    gain , question = find_best_split(rows)
    
    #Base case: no further info gain
    #since we can ask no further questions,
    #we will return a leaf
    
    if gain == 0:
        return Leaf(rows)
    
    # if we reach here, we found a useful feature / value to partition on.
    
    true_rows, false_rows = partition(rows, question)
    
    # recursivey build the true branch and false true
    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    # return a question node
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    #depending on the answer
    
    
def print_tree(node, spacing="" ):
    """ world's most elegant tree printing function."""
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print( spacing + "Predict" , node.predictions)
        return
    
    #print the question at this node
    print(spacing + "--> True:")
    print_tree(node.true_branch, spacing + " ")
    
    # call this function recursively as the false branch
    print(spacing + "-->False:")
    print_tree(node.false_branch, spacing + " ")
    
def classify(row, node):
    
    # base case: we've reached a lead
    if isinstance(node, Leaf):
        return node.predictions

   #Decide whether to follow the true-branch or the false-branch.
   # compare the feature / value stored in the node
   # to the example we're considering

    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    
    
def print_leaf(counts):
    """print the predictions at a leaf"""
    total = sum(counts.values())*1.0
    
    probs={}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
        
    return probs

if __name__ == '__main__':
    my_tree = build_tree(training_data)
    
    print_tree(my_tree)
    
    # evaluate
    testing_data=[
        ['Green', 3, "Mango"],
        ['Yellow', 4, 'Mango'],
        ['Red',2, 'Grape'],
        ['Red', 3, 'Lemon'],
    ]
    
    for row in testing_data:
        print("Actual: %s. predicted: %s" %
             (row[-1], print_leaf(classify(row, my_tree))))
      
    
    
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 222)